# Optimization Routines

In [1]:
using JuMP
using Ipopt

Utility functions to keep probability vector indices in order

In [2]:
function problist_to_dict(p)
    # This function inverts probdict_to_list
    #
    # Input:
    #   p : 16-component probability vector
    #       probabilities of ((A1,S1),(A2,S2)) ordered lexicographically i.e. ((0,0),(0,0)), ((0,0),(0,1)), ..., ((1,1),(1,1))
    # Output:
    #   d : a dictionary whose keys are joint configurations of (X^t,X^{t+1}) (words in A^(2I))
    #       and whose values are the corresponding probabilities
    d=Dict()
    for i in eachindex(p)
        (a,b,c,f) = base(2,i-1,4)
        d[((parse(Int,a),parse(Int,b)),(parse(Int,c),parse(Int,f)))] = p[i]
    end
    return d
end

function probdict_to_list(d)
    # This function inverts problist_to_dict
    #
    # Input:
    #   d : a dictionary whose keys are joint configurations of (X^t,X^{t+1}) (words in A^(2I))
    #       and whose values are the corresponding probabilities
    #   m : alphabet size
    # Output:
    #   p : list of probabilities
    l = [((0,0),(0,0)),
        ((0,0),(0,1)),
        ((0,0),(1,0)),
        ((0,0),(1,1)),
        ((0,1),(0,0)),
        ((0,1),(0,1)),
        ((0,1),(1,0)),
        ((0,1),(1,1)),
        ((1,0),(0,0)),
        ((1,0),(0,1)),
        ((1,0),(1,0)),
        ((1,0),(1,1)),
        ((1,1),(0,0)),
        ((1,1),(0,1)),
        ((1,1),(1,0)),
        ((1,1),(1,1))]
    p = []
    for k in l
        push!(p,d[k])
    end
    return p
end

probdict_to_list (generic function with 1 method)

The main attraction

In [3]:
function integrated_information(Q)
    # Input
    #  Q : 16-component probability vector
    #      probabilities of ((A1,S1),(A2,S2)) ordered lexicographically i.e. ((0,0),(0,0)), ((0,0),(0,1)), ..., ((1,1),(1,1))
    # Output:
    #  Dmin : number, equal to the integrated information
    #  q : 16-component array, the minimizer of Dkl over the null manifold
    
    # create a JuMP model
    m = Model(solver=IpoptSolver())
    
    # define the variable to optimize over - a 16-component nonnegative vector
    @defVar(m, x[1:16] >= 0)
    
    # initialize x to the input distribution
    for i in 1:16
        setValue(x[i],Q[i])
    end
    
    # constrain sum of entries of x to be 1
    unitsum = AffExpr(x, ones(16), 0)
    @addConstraint(m, unitsum == 1)

    # impose constraints derived from conditional probability
    @addNLConstraint(m, (x[1]+x[2])*(x[8]+x[7])- (x[6]+x[5])*(x[4]+x[3]) == 0)
    @addNLConstraint(m, (x[10]+x[9])*(x[16]+x[15])- (x[14]+x[13])*(x[12]+x[11]) == 0)
    @addNLConstraint(m, (x[1]+x[3])*(x[10]+x[12])- (x[9]+x[11])*(x[2]+x[4]) == 0)
    @addNLConstraint(m, (x[5]+x[7])*(x[16]+x[14])- (x[8]+x[6])*(x[13]+x[15]) == 0)

    # build up Dkl objective function using auxillary variables, due to syntax constraints
    @defVar(m, logquo[1:16]) # variable to hold logs of quotients
    for i = 1:16
        @addNLConstraint(m, logquo[i] == log2(x[i]/Q[i]))
    end
    Dkl = QuadExpr(x,logquo,ones(16),0)
    @setObjective(m, Min, Dkl)
    solve(m)
    q = getValue(x)
    Dmin = getObjectiveValue(m)
    return (Dmin,q)
end

integrated_information (generic function with 1 method)

Calculator for stochastic interaction, the information quantity whose null model is obtained from the "Multiplex Markov Chain" framework

In [4]:
function stochastic_interaction(p, p_null)
    # Input
    #  p : 16-component probability vector
    #      probabilities of ((A1,S1),(A2,S2)) ordered lexicographically i.e. ((0,0),(0,0)), ((0,0),(0,1)), ..., ((1,1),(1,1))
    #  p_null : 16-component vector taken from the Multiplex Markov Chain null model
    #      gives *conditional* probabilities
    # Output
    #  Dmin : number, equal to stochastic interaction
    #  q : 16-component probability vector
    
    full = problist_to_dict(p)
    null_conditionals = problist_to_dict(p_null)
    # find probabilities of X^t to convert conditionals to joints
    init_probs = Dict()
    null_SI = Dict()
    
    # marginalize full distribution to obtain p(X^t)
    for (fs,ts) in keys(full)
        if haskey(init_probs,fs)
            init_probs[fs] += full[(fs,ts)]
        else
            init_probs[fs] = full[(fs,ts)]
        end
    end
    
    # convert conditional probabilities to a proper joint distribution for the null model
    for (fs,ts) in keys(null_conditionals)
        null_SI[(fs,ts)] = init_probs[fs]*null_conditionals[(fs,ts)]/sum(values(init_probs))
    end
    
    q = probdict_to_list(null_SI)
    Dmin = sum([x*log2(x/y) for (x,y) in zip(p,q)])
    return (Dmin,q)
end

stochastic_interaction (generic function with 1 method)

Calculator for mutual information between past and present

In [5]:
function mutual_information(p)
    # Input
    #  p : 16-component probability vector
    #      probabilities of ((A1,S1),(A2,S2)) ordered lexicographically i.e. ((0,0),(0,0)), ((0,0),(0,1)), ..., ((1,1),(1,1))
    # Output
    #  MI : mutual information between (A1,S1)  and (A2,S2)
    full = problist_to_dict(p)
    X_marg = Dict()
    Y_marg = Dict()
    null = Dict()
    
    # marginalize the full distribution to obtain X_marg = p(X^t) and Y_marg = p(X^t+1)
    for (fs,ts) in keys(full)
        if fs in keys(X_marg)
            X_marg[fs] += full[(fs,ts)]
        else
            X_marg[fs] = full[(fs,ts)]
        end
        if ts in keys(Y_marg)
            Y_marg[ts] += full[(fs,ts)]
        else
            Y_marg[ts] = full[(fs,ts)]
        end
    end
    
    # obtain null model as product of marginals
    for (fs,ts) in keys(full)
        null[(fs,ts)] = X_marg[fs]*Y_marg[ts]
    end
    
    q = probdict_to_list(null)
    return sum([x*log2(x/y) for (x,y) in zip(p,q)])
end

mutual_information (generic function with 1 method)

In [6]:
function Dkl_oracle(P,Q)
    # Utility function for calculating Kullback-Leibler divergence and its first two derivatives
    # Input
    #  P,Q : two probability vectors of the same length
    # Output
    #  f : Float : D_KL (P||Q)
    #  g : Vector : gradient of D_KL with respect to Q
    #  H : Vector : hessain of D_KL with respect to Q = diag(H)
    nonneg = (i->(i>=0))
    if ~all(nonneg,[P;Q])
        ArgumentError("Probabilities must be nonnegative")
    elseif ~all(i->(i==1),[sum(P),sum(Q)])
        ArgumentError("Probabilities must sum to 1")
    elseif any(x->((x[2]==0) & (x[1]>0)),zip(P,Q))
        ArgumentError("P must be absolutely continuous wrt Q; q==0 -> p==0")
    else
        f=0
        g=[]
        H=[]
        for (p,q) in zip(P,Q)
            f+=p*log2(p/q)
            push!(g,-p/q)
            push!(H,p/q^2)
        end
        return (f,g,H)
    end
end

Dkl_oracle (generic function with 1 method)

# Data
As an example system, I'll look at the edge dynamics of the multi-layer network defined by aggression and status signalling interactions in a monkey society.

The data used are a series of seven weekly snapshots of group activity of the form (giver, receiver, frequency) for each type of interaction. Thus, we can describe each snapshot by a two-layer network, where existence of an edge in one layer signifies presence of an aggressive interaction, and existence of an edge in the other layer signifies presence of a status-signalling interaction. For simplicity, we do not encode directionality in these networks.

Then, each dyad (pair of individuals) can be assigned a "state", which is a binary vector $X$ of length two. The first (second) component of the vector indicates existence of an edge in the first (second) layer. From here, we investigate the dynamics of this vector. We do this by computing the joint probability $p(X^t,X^{t+1})$, by counting, for each possible pair $(x,y)$ of binary vectors, the number of dyads having $X^t = x$ and $X^{t+1}=y$ for any $t\in \{1,\dots,6\}$

This distribution constitutes our "full" model. We can then investigate the extent to which the two components of the binary state vector interact with each other through time.

## Reference point: Stochastic Interaction

Below are data generated from a routine that explicitly computes a null model having the form of a product of marginals of $p$. Explicitly, the null model is defined by $$q(x,y) = p(x) \prod_i p(y_i \vert x_i)$$

The dictionaries named "counts" contain the number of dyads that were observed to undergo the corresponding transition. For instance, the entry $((0,1),(1,0)) \to 133$ means that 133 dyads were observed to exhibit status signalling but no aggression during one week, and aggression but no status signalling the next week.

In [32]:
# Agg/Status, undirected, pre-perturbation

counts_ASundir_pre = Dict([(((0, 1), (1, 0)), 133),
 (((0, 1), (0, 1)), 54),
 (((0, 0), (0, 0)), 17920),
 (((1, 0), (1, 0)), 1023),
 (((0, 0), (0, 1)), 591),
 (((1, 0), (0, 1)), 148),
 (((1, 0), (1, 1)), 73),
 (((0, 0), (1, 1)), 198),
 (((1, 0), (0, 0)), 3378),
 (((1, 1), (0, 1)), 24),
 (((1, 1), (0, 0)), 205),
 (((0, 0), (1, 0)), 3196),
 (((1, 1), (1, 0)), 73),
 (((0, 1), (0, 0)), 605),
 (((0, 1), (1, 1)), 16),
 (((1, 1), (1, 1)), 15)])

tot = sum(values(counts_ASundir_pre))

probs_ASundir_pre = Dict{Any,Any}(zip(keys(counts_ASundir_pre),collect(values(counts_ASundir_pre))/tot))

null_conditionals = Dict([(((0, 1), (0, 1)), 0.082376018098575365),
 (((0, 1), (1, 0)), 0.14079200993913821),
 (((1, 0), (1, 0)), 0.23069025538330704),
 (((0, 0), (0, 1)), 0.032163425922733767),
 (((1, 0), (0, 1)), 0.028969500401011476),
 (((1, 0), (1, 1)), 0.0091397385450475502),
 (((1, 1), (0, 0)), 0.68597417584282472),
 (((1, 0), (0, 0)), 0.73120050567063399),
 (((1, 1), (0, 1)), 0.074195830228820758),
 (((1, 1), (1, 0)), 0.21642156506223301),
 (((1, 1), (1, 1)), 0.023408428866121565),
 (((0, 0), (1, 1)), 0.0059458130233252561),
 (((0, 1), (0, 0)), 0.76160373096591949),
 (((0, 1), (1, 1)), 0.015228240996366965),
 (((0, 0), (1, 0)), 0.15007443791217992),
 (((0, 0), (0, 0)), 0.81181632314176111)])

p = probdict_to_list(probs_ASundir_pre)
p_null = probdict_to_list(null_conditionals)

println("Stochastic Interaction")
println(stochastic_interaction(p,p_null)[1])
println("Integrated Information")
println(integrated_information(p)[1])
println("Mutual Information")
println(mutual_information(p))

Stochastic Interaction
0.002313503019973237
Integrated Information
0.0006018815667706807
Mutual Information
0.007185625749994663


In [9]:
# Agg/Status, undirected, post-perturbation

counts_ASundir_post = Dict([(((0, 1), (1, 0)), 131),
 (((0, 1), (0, 1)), 49),
 (((0, 0), (0, 0)), 10148),
 (((1, 0), (1, 0)), 853),
 (((0, 0), (0, 1)), 486),
 (((1, 0), (0, 1)), 137),
 (((1, 0), (1, 1)), 87),
 (((0, 0), (1, 1)), 177),
 (((1, 0), (0, 0)), 2172),
 (((1, 1), (0, 1)), 32),
 (((1, 1), (0, 0)), 162),
 (((0, 0), (1, 0)), 2198),
 (((1, 1), (1, 0)), 75),
 (((0, 1), (0, 0)), 425),
 (((0, 1), (1, 1)), 27),
 (((1, 1), (1, 1)), 22)])


tot = sum(values(counts_ASundir_post))

probs_ASundir_post = Dict{Any,Any}(zip(keys(counts_ASundir_post),collect(values(counts_ASundir_post))/tot))

null_conditionals = Dict([(((0, 1), (0, 1)), 0.11531734916034557),
 (((0, 1), (1, 0)), 0.15943200255155102),
 (((1, 0), (1, 0)), 0.27705030409987896),
 (((0, 0), (0, 1)), 0.044469015239151879),
 (((1, 0), (0, 1)), 0.03860807890969932),
 (((1, 0), (1, 1)), 0.016004467215761937),
 (((1, 1), (0, 0)), 0.60682649900041208),
 (((1, 0), (0, 0)), 0.66833714977465986),
 (((1, 1), (0, 1)), 0.10011872968394708),
 (((1, 1), (1, 0)), 0.25155187937796636),
 (((1, 1), (1, 1)), 0.041502891937674549),
 (((0, 0), (1, 1)), 0.010143530886309376),
 (((0, 1), (0, 0)), 0.69894637582682739),
 (((0, 1), (1, 1)), 0.026304272461276052),
 (((0, 0), (1, 0)), 0.17559274412651771),
 (((0, 0), (0, 0)), 0.769794709748021)])

p = probdict_to_list(probs_ASundir_post)
p_null = probdict_to_list(null_conditionals)

println("Stochastic Interaction")
println(stochastic_interaction(p,p_null)[1])
println("Integrated Information")
println(integrated_information(p)[1])
println("Mutual Information")
println(mutual_information(p))

Stochastic Interaction
0.003737415222098175
Integrated Information
0.0016622523978824385
Mutual Information
0.01266331318070865


The data below consider probability distributions obtained from particular subsets of the data. In particular, we tabulate week-to-week transition counts observed within:
1. A baseline period of three weeks
2. The three-week period immediately following a social perturbation in which 19 of ~100 individuals were removed from the group
3. The following three-week period

For each of the periods, we calculate integrated information, stochastic interaction, and mutual information, and display the numbers.

In [40]:
counts_ASundir_base3 = Dict([(((0, 1), (1, 0)), 60),
  (((0, 1), (0, 1)), 20),
  (((0, 0), (0, 0)), 8574),
  (((1, 0), (1, 0)), 596),
  (((0, 0), (0, 1)), 290),
  (((1, 0), (0, 1)), 68),
  (((1, 0), (1, 1)), 38),
  (((0, 0), (1, 1)), 113),
  (((1, 0), (0, 0)), 1735),
  (((1, 1), (0, 1)), 9),
  (((1, 1), (0, 0)), 93),
  (((0, 0), (1, 0)), 1686),
  (((1, 1), (1, 0)), 37),
  (((0, 1), (0, 0)), 251),
  (((0, 1), (1, 1)), 6),
  (((1, 1), (1, 1)), 9)])

null_cond_base3 = Dict([(((0, 1), (0, 1)), 0.076730495519388472),
  (((0, 1), (1, 0)), 0.15393355772163134),
  (((1, 0), (1, 0)), 0.25299259115109857),
  (((0, 0), (0, 1)), 0.032323399603369322),
  (((1, 0), (0, 1)), 0.028678672769221682),
  (((1, 0), (1, 1)), 0.010246682138426007),
  (((1, 1), (0, 0)), 0.66868222013558554),
  (((1, 0), (0, 0)), 0.70808205394125379),
  (((1, 1), (0, 1)), 0.068078506574889924),
  (((1, 1), (1, 0)), 0.23891531579870609),
  (((1, 1), (1, 1)), 0.02432395749081849),
  (((0, 0), (1, 1)), 0.006601955304278366),
  (((0, 1), (0, 0)), 0.75366397821266018),
  (((0, 1), (1, 1)), 0.015671968546319932),
  (((0, 0), (1, 0)), 0.16300357096367291),
  (((0, 0), (0, 0)), 0.79807107412867928)])

tot = sum(values(counts_ASundir_base3))
probs_base3 = Dict{Any,Any}(zip(keys(counts_ASundir_base3),collect(values(counts_ASundir_base3))/tot))

counts_ASundir_post3 = Dict([(((0, 1), (1, 0)), 64),
  (((0, 1), (0, 1)), 28),
  (((0, 0), (0, 0)), 6276),
  (((1, 0), (1, 0)), 503),
  (((0, 0), (0, 1)), 232),
  (((1, 0), (0, 1)), 73),
  (((1, 0), (1, 1)), 44),
  (((0, 0), (1, 1)), 94),
  (((1, 0), (0, 0)), 1178),
  (((1, 1), (0, 1)), 20),
  (((1, 1), (0, 0)), 70),
  (((0, 0), (1, 0)), 1426),
  (((1, 1), (1, 0)), 42),
  (((0, 1), (0, 0)), 216),
  (((0, 1), (1, 1)), 16),
  (((1, 1), (1, 1)), 10)])

null_cond_post3 = Dict([(((0, 1), (0, 1)), 0.12954411520966466),
  (((0, 1), (1, 0)), 0.16093242604494701),
  (((1, 0), (1, 0)), 0.29500194278773068),
  (((0, 0), (0, 1)), 0.036519102954343553),
  (((1, 0), (0, 1)), 0.031219152743354597),
  (((1, 0), (1, 1)), 0.013957892433690579),
  (((1, 1), (0, 0)), 0.58029654862816549),
  (((1, 0), (0, 0)), 0.65982101203522425),
  (((1, 1), (0, 1)), 0.11074361615041327),
  (((1, 1), (1, 0)), 0.25944704111542427),
  (((1, 1), (1, 1)), 0.049512794105996995),
  (((0, 0), (1, 1)), 0.0086579422227016194),
  (((0, 1), (0, 0)), 0.67881116369864281),
  (((0, 1), (1, 1)), 0.030712295046745615),
  (((0, 0), (1, 0)), 0.18298677886899101),
  (((0, 0), (0, 0)), 0.77183617595396392)])

tot = sum(values(counts_ASundir_post3))
probs_post3 = Dict{Any,Any}(zip(keys(counts_ASundir_post3),collect(values(counts_ASundir_post3))/tot))

counts_ASundir_last3 = Dict([(((0, 1), (1, 0)), 86),
  (((0, 1), (0, 1)), 28),
  (((0, 0), (0, 0)), 6083),
  (((1, 0), (1, 0)), 474),
  (((0, 0), (0, 1)), 316),
  (((1, 0), (0, 1)), 78),
  (((1, 0), (1, 1)), 53),
  (((0, 0), (1, 1)), 113),
  (((1, 0), (0, 0)), 1345),
  (((1, 1), (0, 1)), 15),
  (((1, 1), (0, 0)), 109),
  (((0, 0), (1, 0)), 1214),
  (((1, 1), (1, 0)), 51),
  (((0, 1), (0, 0)), 296),
  (((0, 1), (1, 1)), 18),
  (((1, 1), (1, 1)), 13)])

null_cond_last3 = Dict([(((0, 1), (0, 1)), 0.10005142440850028),
  (((0, 1), (1, 0)), 0.15426846398148722),
  (((1, 0), (1, 0)), 0.26059991540694472),
  (((0, 0), (0, 1)), 0.047788976702594514),
  (((1, 0), (0, 1)), 0.041930923396932634),
  (((1, 0), (1, 1)), 0.016035598611746846),
  (((1, 1), (0, 0)), 0.63557753379911086),
  (((1, 0), (0, 0)), 0.68143356258437582),
  (((1, 1), (0, 1)), 0.08778695218219762),
  (((1, 1), (1, 0)), 0.24306324289991835),
  (((1, 1), (1, 1)), 0.033572271118773249),
  (((0, 0), (1, 1)), 0.010177545306084971),
  (((0, 1), (0, 0)), 0.72437231271754199),
  (((0, 1), (1, 1)), 0.021307798892470611),
  (((0, 0), (1, 0)), 0.16539871756787286),
  (((0, 0), (0, 0)), 0.7766347604234477)])

tot = sum(values(counts_ASundir_last3))
probs_last3 = Dict{Any,Any}(zip(keys(counts_ASundir_last3),collect(values(counts_ASundir_last3))/tot))

pb = probdict_to_list(probs_base3)
pp = probdict_to_list(probs_post3)
pl = probdict_to_list(probs_last3)
ncb = probdict_to_list(null_cond_base3)
ncp = probdict_to_list(null_cond_post3)
ncl = probdict_to_list(null_cond_last3)

println("Stochastic Interaction")
println([stochastic_interaction(pb,ncb)[1],stochastic_interaction(pp,ncp)[1],stochastic_interaction(pl,ncl)[1]])
println("Integrated Information")
println([integrated_information(pb)[1],integrated_information(pp)[1],integrated_information(pl)[1]])
println("Mutual Information")
println([mutual_information(pb),mutual_information(pp),mutual_information(pl)])

Stochastic Interaction
[0.0025836113515921274,0.00401887075693457,0.00467322430132345]
Integrated Information
[0.0003618016808804067,0.00208207863639009,0.0015264134559528748]
Mutual Information
[0.007582052333549238,0.01589110830795942,0.01056125167764734]
